In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [15]:
외국인역대성적 = pd.read_csv('kbo_yearly_foreigners_2011_2018.csv')
외국인메이저성적 = pd.read_csv('fangraphs_foreigners_2011_2018.csv')
외국인스탯캐스터 = pd.read_csv('baseball_savant_foreigners_2011_2018.csv')
신규외국인성적 = pd.read_csv('fangraphs_foreigners_2019.csv')
신규외국인스텟캐스터 = pd.read_csv('baseball_savant_foreigners_2019.csv')

df = pd.read_csv('최종외국인스탯캐스터.csv',encoding='euc-kr',engine="python")

In [16]:
throw_df = pd.read_csv('plus_win_lose_2.csv')
throw_df = throw_df.fillna(0)
print(throw_df.shape)
top20 = throw_df.sort_values(['승률','경기수'], ascending=False).reset_index(drop=True)[:20]
# bottom20 = throw_df.sort_values(['승률','경기수'], ascending=False).reset_index(drop=True)[-20:]
# display(top20)
# display(bottom20)

(105, 16)


In [4]:
# throw_df.drop('year',axis=1,inplace=True)

### 머신러닝

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [7]:
throw_df['year_born'] = throw_df['year_born'].fillna(0)
throw_df['방출연도'] = throw_df['방출연도'].fillna(0)

In [18]:
throw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 16 columns):
pitcher_name    105 non-null object
year            105 non-null int64
team            105 non-null object
ERA             105 non-null float64
TBF             105 non-null int64
H               105 non-null int64
HR              105 non-null int64
BB              105 non-null int64
HBP             105 non-null int64
SO              105 non-null int64
year_born       105 non-null object
승               105 non-null int64
패               105 non-null int64
승률              105 non-null float64
경기수             105 non-null int64
방출연도            105 non-null float64
dtypes: float64(3), int64(10), object(3)
memory usage: 13.2+ KB


In [9]:
from sklearn.preprocessing import LabelEncoder

def label(df):
    col = list(df)
    encoder = LabelEncoder()
    encoder.fit(col)
    col_label = encoder.transform(col)
    return col_label

pitcher_name_label = label(throw_df['pitcher_name'])
team_label = label(throw_df['team'])
year_born_label = label(throw_df['year_born'])

In [7]:
throw_df = throw_df.assign(pitcher_name_label=pitcher_name_label,
                          team_label=team_label,
                          year_born_label=year_born_label)
throw_df_digit = throw_df.drop(['pitcher_name','team','year_born','year'],axis=1)
throw_df_digit.head()

NameError: name 'throw_df' is not defined

In [26]:
feature = throw_df_digit.drop('승',axis=1)
label = throw_df_digit['승']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(feature,label, test_size = 0.2, random_state=20)

In [28]:
base_dtc = DecisionTreeClassifier(random_state=20, criterion='entropy')

In [29]:
base_dtc.fit(X_train, y_train)
predition = base_dtc.predict(X_test)

In [30]:
print(' y_test\n', y_test)
print(' prediction\n', predition)

 y_test
 72     12
78     10
31     19
5      14
2      11
46      4
38     13
100     9
93      7
0      11
39      7
60      7
81      5
12      7
98      6
36     10
48     18
23      5
65      2
53     10
92      9
Name: 승, dtype: int64
 prediction
 [11  7 15 10 10  4 14 13  6 10  6  4  5  8  8 10 15  5  3  8  8]


In [31]:
from sklearn.metrics import accuracy_score
print(f'예측 정확도 : {accuracy_score(y_test, predition)}')

예측 정확도 : 0.19047619047619047


### 투수 - 외국 경기 평균 변환

#### 외국인메이저성적에 평균 구속과 구종 개수 추가

In [17]:
외국인메이저성적.head()

,pitcher_name,year,ERA,WAR,TBF,H,HR,BB,HBP,SO,WHIP,BABIP,FIP,LD%,GB%,FB%,IFFB%,SwStr%,Swing%
0,오간도,2011.0,3.51,3.3,693.0,149.0,16.0,43.0,7.0,126.0,1.14,0.265,3.65,0.237,0.364,0.674,0.147,0.090,0.475
1,험버,2011.0,3.75,3.2,676.0,151.0,14.0,41.0,6.0,116.0,1.18,0.275,3.58,0.168,0.471,0.458,0.094,0.092,0.463
2,루카스,2012.0,3.76,2.8,827.0,185.0,13.0,78.0,1.0,140.0,1.36,0.289,3.75,0.203,0.572,0.707,0.082,0.062,0.424
3,다이아몬드,2012.0,3.54,2.2,714.0,184.0,17.0,31.0,4.0,90.0,1.24,0.292,3.94,0.210,0.534,0.597,0.040,0.068,0.467
4,듀브론트,2013.0,4.32,2.2,705.0,161.0,13.0,71.0,5.0,139.0,1.43,0.310,3.78,0.199,0.456,0.633,0.127,0.077,0.434


In [18]:
평균구속 = df.groupby('pitcher_name',as_index=False).agg({'km변환':'mean'})\
.sort_values('km변환',ascending=False).reset_index(drop=True)

In [19]:
for idx, value in 평균구속.iterrows():
    for i in range(외국인메이저성적.shape[0]):
        if 외국인메이저성적.loc[i,'pitcher_name'] == value[0]:
            외국인메이저성적.loc[i,'평균구속'] = value[1]

In [20]:
# 외국인메이저성적[외국인메이저성적['평균구속'].isna() == True]

In [21]:
# for val in 외국인메이저성적[외국인메이저성적['평균구속'].isna() == True].drop_duplicates(['pitcher_name'])['pitcher_name']:
#     display(df[df['pitcher_name'] == val])

In [22]:
외국인메이저성적_not_na = 외국인메이저성적[외국인메이저성적['평균구속'].isna() == False].reset_index(drop=True)

외국인메이저성적_drop_year = 외국인메이저성적.drop('year', axis=1)

외국인메이저_평균 = 외국인메이저성적_drop_year.groupby('pitcher_name').agg('mean').reset_index(drop=False).round(decimals=2)

In [23]:
### 구종 개수 추가
playername = list(throw_df['pitcher_name'].drop_duplicates())

구종데이터 = pd.DataFrame({'pitcher_name':[], '2-Seam Fastball':[],'4-Seam Fastball':[],'Changeup':[],'Curveball':[],
                        'Cutter':[],'Intentional Ball':[],'Pitch Out':[],'Slider':[]})
for name in playername:
    try:
        a = 외국인스탯캐스터.groupby(['pitcher_name','pitch_name']).agg({'pitch_name':'count'})\
    .T[name].reset_index(drop=True)
        a['pitcher_name'],a['cnt'] = name, a.shape[1]
        구종데이터 = 구종데이터.append(a,ignore_index=True)
    except:
        print(name)
display(구종데이터.head())
print(구종데이터.cnt.mean())

리즈
카스티요


,pitcher_name,2-Seam Fastball,4-Seam Fastball,Changeup,Curveball,Cutter,Intentional Ball,Pitch Out,Slider,cnt,Sinker,Fastball,Unknown,Split Finger,Forkball,Eephus
0,니퍼트,26.0,384.0,81.0,82.0,6.0,4.0,1.0,4.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
1,소사,19.0,245.0,8.0,NaN,20.0,4.0,2.0,306.0,8.0,269.0,NaN,NaN,NaN,NaN,NaN
2,탈보트,58.0,433.0,429.0,NaN,171.0,9.0,5.0,284.0,8.0,1360.0,NaN,NaN,NaN,NaN,NaN
3,레이예스,773.0,950.0,297.0,88.0,NaN,NaN,3.0,256.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
4,세든,207.0,324.0,182.0,25.0,NaN,1.0,NaN,153.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN


6.3


In [24]:
투수이름 = 구종데이터['pitcher_name']
구종데이터 = 구종데이터.loc[:,구종데이터.columns != 'pitcher_name'].fillna(0).astype(int)

구종데이터['pitcher_name'] = 투수이름

In [25]:
# 구종데이터.head()

In [26]:
for idx, value in 구종데이터.iterrows():
    for idx2, value2 in 외국인메이저_평균.iterrows():
        if value['pitcher_name'] == value2['pitcher_name']:
            외국인메이저_평균.loc[idx2, 'cnt'] = value['cnt']

In [27]:
외국인메이저_평균.head()

,pitcher_name,ERA,WAR,TBF,H,HR,BB,HBP,SO,WHIP,BABIP,FIP,LD%,GB%,FB%,IFFB%,SwStr%,Swing%,평균구속,cnt
0,니퍼트,4.29,-0.30,262.00,61.0,7.00,34.00,5.00,47.00,1.68,0.32,5.09,0.27,0.32,0.67,0.10,0.08,0.42,143.88,8.0
1,다이아몬드,10.26,0.55,369.50,100.0,10.25,21.50,1.25,40.25,2.12,0.33,5.66,0.21,0.47,0.64,0.04,0.06,0.44,139.50,6.0
2,듀브론트,5.11,0.62,377.67,90.0,10.50,36.50,2.33,73.67,1.56,0.32,4.55,0.21,0.44,0.63,0.10,0.08,0.43,141.69,7.0
3,레나도,7.31,-0.50,131.33,31.0,7.33,14.67,0.33,14.67,1.63,0.26,6.70,0.17,0.35,0.62,0.10,0.05,0.45,141.20,4.0
4,레온,6.05,-0.10,63.00,16.5,2.00,5.00,0.00,10.50,1.58,0.30,6.24,0.16,0.48,0.59,0.03,0.10,0.47,140.06,5.0


In [28]:
throw_df_copy = throw_df.copy()
throw_df_copy = throw_df_copy[['pitcher_name','year','승','패']]

In [29]:
외국인_승패_추가 = throw_df_copy.merge(외국인메이저_평균, how='left', on='pitcher_name')
외국인_승패_추가.head()

,pitcher_name,year,승,패,ERA,WAR,TBF,H,HR,BB,...,BABIP,FIP,LD%,GB%,FB%,IFFB%,SwStr%,Swing%,평균구속,cnt
0,리즈,2011,11,13,4.24,-0.2,106.0,26.0,4.0,12.0,...,0.37,4.98,0.35,0.35,0.69,0.16,0.10,0.46,NaN,NaN
1,니퍼트,2011,15,6,4.29,-0.3,262.0,61.0,7.0,34.0,...,0.32,5.09,0.27,0.32,0.67,0.10,0.08,0.42,143.88,8.0
2,니퍼트,2012,11,10,4.29,-0.3,262.0,61.0,7.0,34.0,...,0.32,5.09,0.27,0.32,0.67,0.10,0.08,0.42,143.88,8.0
3,리즈,2012,5,12,4.24,-0.2,106.0,26.0,4.0,12.0,...,0.37,4.98,0.35,0.35,0.69,0.16,0.10,0.46,NaN,NaN
4,소사,2012,9,8,5.23,0.2,231.0,54.0,7.0,23.0,...,0.29,4.77,0.18,0.44,0.62,0.15,0.09,0.45,144.06,8.0


In [30]:
외국인_승패_추가_승 = 외국인_승패_추가['승']
외국인_승패_추가_패 = 외국인_승패_추가['패']

외국인_승패_추가_승패제거 = 외국인_승패_추가.drop(['승','패'],axis=1)

In [44]:
외국인_승패_추가_승패제거[외국인_승패_추가_승패제거['ERA'].isnull()]

,pitcher_name,year,ERA,WAR,TBF,H,HR,BB,HBP,SO,...,BABIP,FIP,LD%,GB%,FB%,IFFB%,SwStr%,Swing%,평균구속,cnt
0,리즈,2011,4.24,-0.2,106.0,26.0,4.0,12.0,3.0,27.0,...,0.37,4.98,0.35,0.35,0.69,0.16,0.1,0.46,NaN,NaN
3,리즈,2012,4.24,-0.2,106.0,26.0,4.0,12.0,3.0,27.0,...,0.37,4.98,0.35,0.35,0.69,0.16,0.1,0.46,NaN,NaN
6,리즈,2013,4.24,-0.2,106.0,26.0,4.0,12.0,3.0,27.0,...,0.37,4.98,0.35,0.35,0.69,0.16,0.1,0.46,NaN,NaN
12,밴덴헐크,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,밴덴헐크,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,카스티요,2016,13.50,0.0,8.0,3.0,0.0,1.0,0.0,2.0,...,0.60,2.41,0.60,0.40,0.50,0.00,0.2,0.47,NaN,NaN
58,마리몬,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,브리검,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,브리검,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 의사결정 나무 진행

### 학습 데이터, 테스트 데이터 분리_승리 먼저

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_text, y_train, y_test = train_test_split(외국인_승패_추가_승패제거, 외국인_승패_추가_승,test_size = 0.2, random_state=20)

In [34]:
from sklearn.tree     import DecisionTreeClassifier

In [38]:
외국인_승패_추가_승패제거.head()

,pitcher_name,year,ERA,WAR,TBF,H,HR,BB,HBP,SO,...,BABIP,FIP,LD%,GB%,FB%,IFFB%,SwStr%,Swing%,평균구속,cnt
0,리즈,2011,4.24,-0.2,106.0,26.0,4.0,12.0,3.0,27.0,...,0.37,4.98,0.35,0.35,0.69,0.16,0.10,0.46,NaN,NaN
1,니퍼트,2011,4.29,-0.3,262.0,61.0,7.0,34.0,5.0,47.0,...,0.32,5.09,0.27,0.32,0.67,0.10,0.08,0.42,143.88,8.0
2,니퍼트,2012,4.29,-0.3,262.0,61.0,7.0,34.0,5.0,47.0,...,0.32,5.09,0.27,0.32,0.67,0.10,0.08,0.42,143.88,8.0
3,리즈,2012,4.24,-0.2,106.0,26.0,4.0,12.0,3.0,27.0,...,0.37,4.98,0.35,0.35,0.69,0.16,0.10,0.46,NaN,NaN
4,소사,2012,5.23,0.2,231.0,54.0,7.0,23.0,3.0,38.0,...,0.29,4.77,0.18,0.44,0.62,0.15,0.09,0.45,144.06,8.0


In [35]:
base_dtc = DecisionTreeClassifier(random_state=20, criterion='entropy')
base_dtc.fit(X_train, y_train)

ValueError: could not convert string to float: '산체스'